In [1]:
from azureml.core import Workspace, Datastore, Experiment, Environment
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline

In [2]:
pip install azureml-pipeline

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Initialize workspace
ws = Workspace.from_config("Users/mypersonall3099/Online_retail/config.json")

# Define compute target
compute_name = "OnlineRetail"
compute_target = ws.compute_targets[compute_name]

In [4]:
# Define environment
env = Environment.from_conda_specification(name="myenv", file_path="environment.yaml")
env.register(workspace=ws)


{
    "assetId": "azureml://locations/eastus2/workspaces/7ce76f32-f5fe-4dd3-b900-96fb92b58ebb/environments/myenv/versions/3",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20240709.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "myenv",
    "python": {
       

In [5]:
from azureml.core import Environment

env = Environment.get(workspace=ws, name="myenv")
print(env.python.conda_dependencies.serialize_to_string())


name: myenv
channels:
- defaults
dependencies:
- python=3.7
- scikit-learn
- statsmodels
- pip
- pip:
  - pmdarima
  - azureml-sdk
  - azureml-dataset-runtime[fuse, pandas]
  - joblib
  - scikit-learn



In [10]:
# Define run configuration
aml_config = RunConfiguration()
aml_config.target = compute_target
aml_config.environment = env

In [14]:
#Pipeline
read_data = 'data_wrangling.py'
prep = 'preprocessing.py'
model = 'modelling.py'

source_directory = ""

#Script initialization
py_script_run_read = PythonScriptStep(
                name='Data Wrangling Step',
                script_name = read_data,
                compute_target = compute_target,
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_prep = PythonScriptStep(
                script_name = prep,
                compute_target=compute_target,
                source_directory=source_directory,
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_model = PythonScriptStep(
                script_name = model,
                compute_target=compute_target,
                source_directory=source_directory,
                runconfig = aml_config,
                allow_reuse=False)

pipeline_steps = [py_script_run_read, py_script_run_prep, py_script_run_model]
pipeline_1 = Pipeline(workspace=ws, steps = pipeline_steps)

In [15]:
# Experiment 1

experiment_name = "Run4"
experiment = Experiment(ws, experiment_name)
pipeline_run = experiment.submit(pipeline_1)
pipeline_run.wait_for_completion(show_output=True)

Created step Data Wrangling Step [8cc423b6][e2f523d9-0842-4236-a538-a5fe911eca2c], (This step will run and generate new outputs)
Created step preprocessing.py [dab3d5cb][b5855ced-d93b-43ed-8433-6388a3618ba2], (This step will run and generate new outputs)
Created step modelling.py [9edaf896][0f01cc27-8f9c-4d96-bb3d-4a0f3e217be2], (This step will run and generate new outputs)
Submitted PipelineRun 5f4f96dd-24ae-4241-b617-94ac0b6c9cc9
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/5f4f96dd-24ae-4241-b617-94ac0b6c9cc9?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRunId: 5f4f96dd-24ae-4241-b617-94ac0b6c9cc9
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/5f4f96dd-24ae-4241-b617-94ac0b6c9cc9?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
Pipeline

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Pipeline has failed child jobs. For more details and logs, please go to the job detail page and check the child jobs.",
        "messageFormat": "Pipeline has failed child jobs. {0}",
        "messageParameters": {},
        "referenceCode": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "eastus2",
    "location": "eastus2",
    "time": "2024-08-18T14:37:48.518822Z",
    "componentName": ""
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Pipeline has failed child jobs. For more details and logs, please go to the job detail page and check the child jobs.\",\n        \"messageFormat\": \"Pipeline has failed child jobs. {0}\",\n        \"messageParameters\": {},\n        \"referenceCode\": \"Pipeli

In [13]:
# Experiment 2

pipeline_run = Experiment(ws, "Exp2").submit(pipeline_1)
pipeline_run.wait_for_completion(show_output=True)

Submitted PipelineRun 07e6bc8b-875d-485b-978c-d438a1c8b471
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/07e6bc8b-875d-485b-978c-d438a1c8b471?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRunId: 07e6bc8b-875d-485b-978c-d438a1c8b471
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/07e6bc8b-875d-485b-978c-d438a1c8b471?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 1048dfa3-19f2-4b9d-90dc-ab945944ec81
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/1048dfa3-19f2-4b9d-90dc-ab945944ec81?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
StepRun( Data Wrangling 

'Finished'

In [33]:
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
Second_run,42601c31-040e-4aac-91b3-8ed00e0e80fa,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation
